## Extract

In [2]:
import requests
import bs4 as bs
import pandas as pd

road_signs = "https://e-seimas.lrs.lt/rs/legalact/TAK/aba0b290c53911e3afd8f519d03d5f2d/"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0"}

response = requests.get(road_signs, headers=headers)

soup = bs.BeautifulSoup(response.text, 'lxml')

In [3]:
def extract_data(table):
    data = [[td.img['src'] if td.find('img') else 
            ''.join(td.stripped_strings).replace("\r\n", "")
                for td in row.find_all('td')]
                    for row in table.find_all('tr')]
    return pd.DataFrame(data[1:], columns=data[0])

tables = soup.find_all("table")

ispejamieji, pirmumo, draudziamieji,nukreipiamieji,nurodomieji,informaciniai,paslaugu,papildomi = tables

extract_data(pirmumo)

,Numeris,Pavadinimas,Pavyzdys,Paaiškinimas
0,201,Pagrindinis kelias,content_files/image052.gif,"Kelias, kuriame suteikta pirmenybė važiuoti p..."
1,202,Pagrindinio kelio pabaiga,content_files/image053.gif,Kelio ženklu „Pagrindinis kelias“ pažymėto ke...
2,203,Duoti kelią,content_files/image054.gif,Vairuotojas privalo duoti kelią transporto pr...
3,204,Stop,content_files/image055.gif,Draudžiama važiuoti nesustojus prieš „Stop“ l...
4,205,Priešpriešinio eismo pirmenybė,content_files/image056.gif,"Įvažiuoti į siaurą kelio ruožą draudžiama, je..."
5,206,Pirmenybė priešpriešinio eismo atžvilgiu,content_files/image057.gif,"Siauras kelio ruožas, kuriuo važiuodamas vair..."


In [4]:
import os


asset_dir = "public/assets"

for table in tables:
    for link in extract_data(table)["Pavyzdys"]:
        url = road_signs + link
        filename = url.split("/")[-1]
        response = requests.get(url, headers=headers)

        with open(os.path.join(asset_dir, filename), "wb") as outfile:
            outfile.write(response.content)

## Transform

In [5]:
context = [
    {"type": "Įspėjamieji kelio ženklai",
     "details": [
        "Įspėjamieji kelio ženklai, įspėjantys vairuotoją apie būsimą pavojingą kelio ruožą, kuriuo važiuojant reikia imtis atitinkamų atsargumo priemonių.",
        "Jeigu įspėjamieji kelio ženklai statomi kartu su papildoma lentele „Galiojimo zona į priekį“, ši lentelė nurodo, kokio ilgio yra pavojingas ruožas, kuris prasideda už 150–300 m (ne gyvenvietėse) arba už 50–100 m (gyvenvietėse) nuo įspėjamojo kelio ženklo pastatymo vietos."
    ]},

    {
        "type": "Pirmumo kelio ženklai",
        "details": [
            "Pirmumo kelio ženklai, nurodantys vairuotojui važiavimo per nereguliuojamas sankryžas, važiuojamųjų dalių sankirtas arba siaurus kelio ruožus pirmenybę."
        ]
    },
    {
        "type": "Draudžiamieji kelio ženklai",
        "details": [
            "Draudžiamieji kelio ženklai ką nors draudžia arba tuos draudimus panaikina",
            "Galiojimo zoną turinčių draudžiamųjų kelio ženklų (319, 325, 327, 329, 331–335) draudimai galioja nuo draudžiamojo kelio ženklo iki artimiausios sankryžos už draudžiamojo kelio ženklo, o gyvenvietėse, kai nėra sankryžos, – iki gyvenvietės pabaigos. Draudžiamųjų kelio ženklų galiojimas nesibaigia ties lauko, miško ir kitais šalutiniais keliais, taip pat ties išvažiavimu iš šalia kelio esančios teritorijos, kuriame nėra pirmumo kelio ženklų.",
            "Pastatytas prieš gyvenvietę, pažymėtą nurodomuoju kelio ženklu „Gyvenvietės pradžia“, draudžiamasis kelio ženklas „Ribotas greitis“ galioja iki šio nurodomojo kelio ženklo, tačiau draudžiamojo kelio ženklo galiojimo zona gali būti sumažinta pastačius kitą draudžiamąjį kelio ženklą „Ribotas greitis“ su kitokiu leidžiamu važiavimo greičiu."
        ]
    },
    {
        "type": "Nukreipiamieji kelio ženklai",
        "details": []
    },
    {
        "type": "Nurodomieji kelio ženklai",
        "details": [
            "Nurodomieji kelio ženklai nustato arba panaikina tam tikrą eismo tvarką"
        ]
    },
    {
        "type": "Informaciniai kelio ženklai",
        "details": []
    },
    {
        "type": "Paslaugų kelio ženklai",
        "details": []
    },
    {
        "type": "Papildomos lentelės",
        "details": [
            "Papildomos lentelės (toliau – lentelės) patikslina arba apriboja kelio ženklų, su kuriais jos naudojamos, galiojimą"
        ]
    }
]


In [6]:
dfs = [extract_data(table) for table in tables]

for index, df in enumerate(dfs):
    df = df.rename(columns={"Numeris": "number", "Pavadinimas": "title", "Pavyzdys": "example", "Paaiškinimas": "explanation" })
    df["type"] = context[index]["type"]
    df["example"] = df["example"].str.split("/").str.get(1)
    dfs[index] = df

dfs[1]

,number,title,example,explanation,type
0,201,Pagrindinis kelias,image052.gif,"Kelias, kuriame suteikta pirmenybė važiuoti p...",Pirmumo kelio ženklai
1,202,Pagrindinio kelio pabaiga,image053.gif,Kelio ženklu „Pagrindinis kelias“ pažymėto ke...,Pirmumo kelio ženklai
2,203,Duoti kelią,image054.gif,Vairuotojas privalo duoti kelią transporto pr...,Pirmumo kelio ženklai
3,204,Stop,image055.gif,Draudžiama važiuoti nesustojus prieš „Stop“ l...,Pirmumo kelio ženklai
4,205,Priešpriešinio eismo pirmenybė,image056.gif,"Įvažiuoti į siaurą kelio ruožą draudžiama, je...",Pirmumo kelio ženklai
5,206,Pirmenybė priešpriešinio eismo atžvilgiu,image057.gif,"Siauras kelio ruožas, kuriuo važiuodamas vair...",Pirmumo kelio ženklai


In [7]:
import json

output = []

for index, df in enumerate(dfs):
    signs = json.loads(df.to_json(orient="records"))

    for sign in signs:
        sign["details"] = context[index].get("details", [])
        output.append(sign)

output[0]

{'number': '101',
 'title': 'Pervaža su užtvaru',
 'example': 'image001.gif',
 'explanation': 'Geležinkelio  pervaža su pakeliamuoju užtvaru',
 'type': 'Įspėjamieji kelio ženklai',
 'details': ['Įspėjamieji kelio ženklai, įspėjantys vairuotoją apie būsimą pavojingą kelio ruožą, kuriuo važiuojant reikia imtis atitinkamų atsargumo priemonių.',
  'Jeigu įspėjamieji kelio ženklai statomi kartu su papildoma lentele „Galiojimo zona į priekį“, ši lentelė nurodo, kokio ilgio yra pavojingas ruožas, kuris prasideda už 150–300 m (ne gyvenvietėse) arba už 50–100 m (gyvenvietėse) nuo įspėjamojo kelio ženklo pastatymo vietos.']}

## Load

In [9]:
with open("src/assets/data.json", "w") as outfile:
    json.dump(output, outfile)